In [ ]:
# from tensorflow.keras.appliications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import AUC,Precision,Recall
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomTranslation, RandomZoom
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,Tensorboard

from tensorflow.keras.mixed_precision import set_global_policy
import numpy as np
import os

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except RuntimeError as e:
        print(e)


In [ ]:

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
set_global_policy('mixed_float16')
tf.keras.backend.clear_session()

data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal"),
    RandomTranslation(0.1, 0.1),
    RandomZoom(0.1),
    RandomRotation(0.05)
])
with tf.device('/GPU:0'):
    base_model = EfficientNetB0(include_top=False, weights='imagenet')
    base_model.trainable = True  

    inputs = layers.Input(shape=(224, 224, 1))
    x = data_augmentation(inputs)
    x = layers.Conv2D(3, (3, 3), padding='same')(x)  
    x = base_model(x)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(7, activation='sigmoid', dtype='float32')(x)  
    model = models.Model(inputs, outputs)

    for layer in base_model.layers[:210]:
        layer.trainable = False

    for i, layer in enumerate(base_model.layers):
        if layer.trainable:
            print(i, layer.name, layer.trainable)

model.summary()


In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-5), loss = "binary_crossentropy", metrics = ["binary_accuracy"])

In [ ]:
X_train = np.load("../data/numpy/X_train.npy")
Y_train = np.load("../data/numpy/Y_train.npy")

X_val = np.load("../data/numpy/X_valid.npy")
Y_val = np.load("../data/numpy/Y_valid.npy")

In [ ]:
my_callbacks = [
    ModelCheckpoint(
        monitor = "val_loss" ,# commonly uses valloss why? cuz secret
        verbose =1,
        filepath = "../checkpoint/model2_{epoch:02d}.keras",
        save_best_only = True,
        save_weights_only = False
    ),

    ReduceLROnPlateau(
        monitor = "val_loss",
        factor = 0.5,
        patience = 5,
        verbose = 1,
        min_lr = 1e-7
    )
    # EarlyStopping(
    #     monitor = 'val_loss',
    #     patience = 5,
    #     restore_best_weights =True
    # )
    
]

train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(8)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(8)



In [ ]:
# history = model.fit(
#     train_ds,
#     validation_data=val_ds,
#     # initial_epoch = 31,
#     epochs=25,
#     # batch_size = 8,
#     verbose = 1,
#     callbacks = my_callbacks
# )

In [ ]:
# model = models.load_model("../checkpoint/model2_04.keras")


os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
set_global_policy('mixed_float16')
tf.keras.backend.clear_session()

data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal"),
    RandomTranslation(0.1, 0.1),
    RandomZoom(0.1),
    RandomRotation(0.05)
])
with tf.device('/GPU:0'):
    base_model = EfficientNetB0(include_top=False, weights='imagenet')
    base_model.trainable = True  

    inputs = layers.Input(shape=(224, 224, 1))
    x = data_augmentation(inputs)
    x = layers.Conv2D(3, (3, 3), padding='same')(x)  
    x = base_model(x)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(7, activation='sigmoid', dtype='float32')(x)  
    model = models.Model(inputs, outputs)

    for layer in base_model.layers[:119]:
        layer.trainable = False

    for i, layer in enumerate(base_model.layers):
        if layer.trainable:
            print(i, layer.name, layer.trainable)

model.summary()



In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    # initial_epoch = 31,
    epochs=25,
    # batch_size = 8,
    verbose = 1,
    callbacks = my_callbacks
)

In [ ]:
model.save("../checkpoint/saved2_model.keras")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['binary_accuracy'], label='train_accuracy')
plt.plot(history.history['val_binary_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model = models.load_model("../checkpoint/model2_23.keras")

In [ ]:
model.summary()
print(model.layers[3].summary())

In [ ]:
# https://keras.io/examples/vision/grad_cam/
import cv2
import matplotlib.pyplot as plt

img_size = (224,224)
last_conv_layer_name = "top_conv"
img = cv2.imread("../data/tests/00030786_003.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

plt.imshow(img)
plt.show()


In [ ]:
for layer in model.layers:
    print(layer.name)